<span style='color:darkcyan'><font size="3">__Juan Sebastian Deslarzes__</font></span><br>
<span style='color:darkcyan'><font size="3">__Chloé De Ancos__</font></span><br>
<span style='color:darkcyan'><font size="3">__Anaïs Burrus__</font></span><br>
<span style='color:darkcyan'><font size="3">__Margot Chapot__</font></span><br>
<span style='color:darkcyan'><font size="3">__Eva Sarlin__</font></span><br>

In [1]:
import matplotlib.pyplot as plt 
%matplotlib inline
import pandas as pd
import numpy as np
from skimage import io
from IPython import display
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
import seaborn as sns
sns.set(color_codes=True)

Loading BokehJS ...

In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from collections import Counter 
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import classification_report
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.compose import ColumnTransformer
import string

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/evasarlin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<span style='color:mediumpurple'><font size="6">__Data Loading__</font></span><br>

In [3]:
sample_submission_url = 'https://raw.githubusercontent.com/esarlin/Challenge-2-CDAW/main/sitc-challenge2-2022/sample_submission.csv'
test_nolabel_url= 'https://raw.githubusercontent.com/esarlin/Challenge-2-CDAW/main/sitc-challenge2-2022/test_nolabel.csv'
train_url = 'https://raw.githubusercontent.com/esarlin/Challenge-2-CDAW/main/sitc-challenge2-2022/train.csv'

sample_submission = pd.read_csv(sample_submission_url)
test_nolabel = pd.read_csv(test_nolabel_url)
df = pd.read_csv(train_url)

<span style='color:mediumpurple'><font size="6">__Feature Selection and Cleaning__</font></span><br>

In [4]:
data_features = df.copy()[["statement", "subject", "speaker","speaker_job","state_info", "party_affiliation"]]
test_features = test_nolabel.copy()[["statement", "subject", "speaker","speaker_job","state_info","party_affiliation"]]
target = df.label.values

<span style='color:steelblue'><font size="4">__Removing Nan values in Speaker job and state features :__</font></span><br>

In [5]:
data_features['speaker_job']= data_features['speaker_job'].fillna('u.s. senator')
test_features['speaker_job']= test_features['speaker_job'].fillna('u.s. senator')
data_features['state_info']= data_features['state_info'].fillna('unknown')
test_features['state_info']= test_features['state_info'].fillna('unknown')

<span style='color:mediumpurple'><font size="6">__NLP__</font></span><br>

<span style='color:steelblue'><font size="4">__Defining functions for string processing__</font></span><br>

In [6]:
# Generic Transformer 
class GenericTransformer(BaseEstimator, TransformerMixin):

    def transform(self, X, y=None):
        return do_something_to(X, self.vars)  # where the actual feature extraction happens

    def fit(self, X, y=None):
        return self  # used if the feature requires training, for example, clustering
    
    
    
class LexicalStats (BaseEstimator, TransformerMixin):
    """Extract lexical features from each document"""
    
    def number_sentences(self, doc):
        sentences = sent_tokenize(doc, language='english')
        return len(sentences)

    def fit(self, x, y=None):
        return self

    def transform(self, docs):
        return [{'length': len(doc),
                 'num_sentences': self.number_sentences(doc)}
                for doc in docs]
    
def custom_tokenizer(words):
    """Preprocessing tokens as seen in the lexical notebook"""
    tokens = word_tokenize(words.lower())
    porter = PorterStemmer()
    lemmas = [porter.stem(t) for t in tokens]
    stoplist = stopwords.words('english')
    lemmas_clean = [w for w in lemmas if w not in stoplist]
    punctuation = set(string.punctuation)
    lemmas_punct = [w for w in lemmas_clean if  w not in punctuation]
    return lemmas_punct

class PosStats(BaseEstimator, TransformerMixin):
    """Obtain number of tokens with POS categories"""

    def stats(self, doc):
        tokens = custom_tokenizer(doc)
        tagged = pos_tag(tokens, tagset='universal')
        counts = Counter(tag for word,tag in tagged)
        total = sum(counts.values())
        #copy tags so that we return always the same number of features
        pos_features = {'NOUN': 0, 'ADJ': 0, 'VERB': 0, 'ADV': 0, 'CONJ': 0, 
                        'ADP': 0, 'PRON':0, 'NUM': 0}
        
        pos_dic = dict((tag, float(count)/total) for tag,count in counts.items())
        for k in pos_dic:
            if k in pos_features:
                pos_features[k] = pos_dic[k]
        return pos_features
    
    def transform(self, docs, y=None):
        return [self.stats(doc) for doc in docs]
    
    def fit(self, docs, y=None):
        """Returns `self` unless something different happens in train and test"""
        return self
    
ngrams_featurizer = Pipeline([
  ('count_vectorizer',  CountVectorizer(ngram_range = (1, 3), encoding = 'ISO-8859-1', 
                                        tokenizer=custom_tokenizer)),
  ('tfidf_transformer', TfidfTransformer())
])

In [7]:
transformer= FeatureUnion([
                    ('lexical_stats', Pipeline([
                                ('stats', LexicalStats()),
                                ('vectors', DictVectorizer())
                            ])),
                    ('words', TfidfVectorizer(tokenizer=custom_tokenizer)),
                    ('ngrams', ngrams_featurizer),
                    ('pos_stats', Pipeline([
                                ('pos_stats', PosStats()),
                                ('vectors', DictVectorizer())
                            ])),
                    ('lda', Pipeline([ 
                                ('count', CountVectorizer(tokenizer=custom_tokenizer)),
                                ('lda',  LatentDirichletAllocation(n_components=4, max_iter=5,
                                                       learning_method='online', 
                                                       learning_offset=50.,
                                                       random_state=0))
                            ])),
                ])

<span style='color:steelblue'><font size="4">__Applying the processing techniques on all the features using ColumnTransformer :__</font></span><br>

In [8]:
col_preprocessor = ColumnTransformer(
        [
            ('title_glove', transformer, 'statement'),
            ('description_glove', transformer, 'subject'),
            ('speaker_glove', transformer, 'speaker'),
            ('speakerjob_glove', transformer, 'speaker_job'),
            ('stateinfo_glove', transformer, 'state_info'),
            ('party_glove', transformer, 'party_affiliation'),
],
        remainder='drop',
        n_jobs=1
        )

<span style='color:mediumpurple'><font size="6">__Machine Learning algorithm__</font></span><br>


<span style='color:steelblue'><font size="4">__Classifier insertion into the pipeline:__</font></span><br>

In [9]:
pipeline_glove = Pipeline([
    ('col_preprocessor', col_preprocessor), 
    ('classifier', MultinomialNB(alpha=.01))
])

<span style='color:steelblue'><font size="4">__Training :__</font></span><br>

In [10]:
pipeline_glove.fit(data_features, target)

Pipeline(steps=[('col_preprocessor',
                 ColumnTransformer(n_jobs=1,
                                   transformers=[('title_glove',
                                                  FeatureUnion(transformer_list=[('lexical_stats',
                                                                                  Pipeline(steps=[('stats',
                                                                                                   LexicalStats()),
                                                                                                  ('vectors',
                                                                                                   DictVectorizer())])),
                                                                                 ('words',
                                                                                  TfidfVectorizer(tokenizer=<function custom_tokenizer at 0x7fe8758713a0>)),
                                                   

<span style='color:mediumpurple'><font size="6">__Prediction__</font></span><br>

In [11]:
y_p=pipeline_glove.predict(test_features)

<div class="alert alert-block alert-info">
    <span style='color:darkblue'><font size="4"><b>CONCLUSION ALGORITHMS</b></font></span><br><br> <span style='color:black'> After several machine algorithms, we can conclude that the multinomial gives us the best accuracy (58% against ...). </span>
</div>

<span style='color:mediumpurple'><font size="6">__Submission__</font></span><br>

In [12]:
id = test_nolabel['id']
dfsubmission = pd.DataFrame({'id': id, 'label': y_p}, columns=["id", "label"])
dfsubmission.to_csv('submission58MultinomialColAll.csv', index=False, header=True)

<div class="alert alert-block alert-info">
    <span style='color:purple'><font size="4"><b>CONCLUSION GENERALE</b></font></span><br><br> <span style='color:black'> This methods, by applying complete natural language processing to each features in parallel using a pipeline gives us the best accuracy of 58.2%, using a multinomial classification algorithm. </span>
</div>